In [1]:
import pandas as pd
import numpy as np
import mysql.connector
import re

In [2]:
# upload all files
df1 = pd.read_csv('punks_atributes1.csv')
df2 = pd.read_csv('punks_atributes2.csv')
df3 = pd.read_csv('punks_atributes3.csv')
df4 = pd.read_csv('punks_atributes4.csv')
df5 = pd.read_csv('punks_atributes5.csv')

In [3]:
# merge into a single csv
df = pd.concat([df1,df2,df3,df4,df5],ignore_index=False)
df.drop('Unnamed: 0',axis=1,inplace=True)

In [4]:
# tenemos 10,000 punks
df.shape

(10000, 6)

In [5]:
df.head(10)

,id,types,rarity,skin,total_traits,traits
0,0,Female,2023,Mid,3,"Blonde Bob,Earring,Green Eye Shadow,Hidden Ear..."
1,1,Male,4352,Dark,2,"Mohawk,Smile"
2,2,Female,8090,Light,1,Wild Hair
3,3,Male,7896,Dark,3,"Nerd Glasses,Pipe,Wild Hair"
4,4,Male,7332,Mid,4,"Big Shades,Earring,Goat,Wild Hair"
5,5,Female,1960,Mid,3,"Earring,Half Shaved,Purple Eye Shadow,Hidden E..."
6,6,Male,7425,Mid,1,Do-rag
7,7,Female,866,Mid,3,"Clown Eyes Blue,Spots,Wild White Hair"
8,8,Male,6530,Dark,2,"Luxurious Beard,Messy Hair"
9,9,Male,1761,Mid,3,"Big Beard,Clown Nose,Police Cap"


In [6]:
df.tail()

,id,types,rarity,skin,total_traits,traits
2553,9995,Female,2466,Albino,2,"Purple Eye Shadow,Straight Hair Dark"
2554,9996,Male,4323,Light,4,"Cigarette,Crazy Hair,Earring,Smile"
2555,9997,Zombie,498,Zombie,2,"Cap Forward,Front Beard"
2556,9998,Female,1271,Mid,3,"Black Lipstick,Clown Eyes Green,Wild White Hair"
2557,9999,Female,9233,Dark,2,"Mohawk,Nerd Glasses"


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 2557
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            10000 non-null  int64 
 1   types         10000 non-null  object
 2   rarity        10000 non-null  int64 
 3   skin          10000 non-null  object
 4   total_traits  10000 non-null  int64 
 5   traits        9992 non-null   object
dtypes: int64(3), object(3)
memory usage: 546.9+ KB


In [8]:
# some missing values
df[df.traits.isnull()]

,id,types,rarity,skin,total_traits,traits
181,281,Male,7,Light,0,NaN
410,510,Male,9,Mid,0,NaN
541,641,Female,6,Light,0,NaN
641,741,Male,5,Dark,0,NaN
950,1050,Female,8,Mid,0,NaN
204,2204,Female,4,Dark,0,NaN
307,3307,Male,3,Albino,0,NaN
3487,6487,Female,2,Albino,0,NaN


In [9]:
df.fillna('',inplace=True)

In [10]:
df1 = df.copy()

In [11]:
def get_number_hidden_traits(text):
    pattern = ',Hidden\s(\w+)'
    try:
        hidden = re.findall(pattern,text)
        return len(hidden)    
    except:
        return 0

In [12]:
#examples of regex
text1 = 'Blonde Bob,Earring,Green Eye Shadow,Hidden Earring,,Hidden Eear'
text2 = 'Blonde Bob,Earring,Green Eye Shadow'
text3= 'Earring,Hidden bob'

def extract_hidden_traits(text):
    match = re.findall(',Hidden\s(\w+)',text)
    if not match:
        #print("List is empty")
        return '' 
    else:
        return ','.join(match)


extract_hidden_traits(text3)

'bob'

In [13]:
df1['total_hidden_traits']=df1['traits'].apply(get_number_hidden_traits)

In [14]:
df1['hidden_traits']=df1['traits'].apply(extract_hidden_traits)

In [15]:
text1 = 'Blonde Bob,Earring,Green Eye Shadow,Hidden Earring,Hidden Eear'

def remove_hidden_traits(text):
    match = re.findall(',Hidden\s(\w+)',text)    
    if not match:
        #print("List is empty")
        return text 
    else:
        to_remove= ',Hidden '+',Hidden '.join(match)        
        clean = text.replace(to_remove,'')        
        return clean


remove_hidden_traits(text1)
    

'Blonde Bob,Earring,Green Eye Shadow'

In [16]:
df1['traits'] = df1['traits'].apply(remove_hidden_traits)

In [17]:
df1.traits

0                   Blonde Bob,Earring,Green Eye Shadow
1                                          Mohawk,Smile
2                                             Wild Hair
3                           Nerd Glasses,Pipe,Wild Hair
4                     Big Shades,Earring,Goat,Wild Hair
                             ...                       
2553               Purple Eye Shadow,Straight Hair Dark
2554                 Cigarette,Crazy Hair,Earring,Smile
2555                            Cap Forward,Front Beard
2556    Black Lipstick,Clown Eyes Green,Wild White Hair
2557                                Mohawk,Nerd Glasses
Name: traits, Length: 10000, dtype: object

In [18]:
df1

,id,types,rarity,skin,total_traits,traits,total_hidden_traits,hidden_traits
0,0,Female,2023,Mid,3,"Blonde Bob,Earring,Green Eye Shadow",1,Earring
1,1,Male,4352,Dark,2,"Mohawk,Smile",0,
2,2,Female,8090,Light,1,Wild Hair,0,
3,3,Male,7896,Dark,3,"Nerd Glasses,Pipe,Wild Hair",0,
4,4,Male,7332,Mid,4,"Big Shades,Earring,Goat,Wild Hair",0,
...,...,...,...,...,...,...,...,...
2553,9995,Female,2466,Albino,2,"Purple Eye Shadow,Straight Hair Dark",0,
2554,9996,Male,4323,Light,4,"Cigarette,Crazy Hair,Earring,Smile",0,
2555,9997,Zombie,498,Zombie,2,"Cap Forward,Front Beard",0,
2556,9998,Female,1271,Mid,3,"Black Lipstick,Clown Eyes Green,Wild White Hair",0,


In [19]:
df1.to_csv('raw_punks.csv')

In [20]:
#get traits 
traits ={}
for items in df1.traits.str.split(','):
    for elem in items:        
        if len(elem) ==0:
            #print(type(elem))
            continue            
        if elem in traits:
            #print('exists, add one')
            traits[elem] += 1
        else:
            #print('not exist, create set to one ')
            traits[elem] = 1
    

In [21]:
df_traits = pd.DataFrame(list(traits.items()),columns = ['trait','occurences']) 
df_traits.drop('occurences',axis=1,inplace=True)

In [22]:
df_traits.to_csv('trait.csv')

In [236]:
hidden_trait = []
for row in df1.hidden_traits.str.split(','):
    for elem in row:
        if len(elem) ==0:
            continue
        else: 
            hidden_trait.append(elem)


In [237]:
hidden_trait= list(set(hidden_trait))

In [238]:
df_hidden_trait = pd.DataFrame(hidden_trait,columns=['name'])

In [242]:
df_hidden_trait.to_csv(r'./to_database/hidden_trait.csv',index=True)

In [24]:
df2=df1.copy()

In [25]:
df2.head()

,id,types,rarity,skin,total_traits,traits,total_hidden_traits,hidden_traits
0,0,Female,2023,Mid,3,"Blonde Bob,Earring,Green Eye Shadow",1,Earring
1,1,Male,4352,Dark,2,"Mohawk,Smile",0,
2,2,Female,8090,Light,1,Wild Hair,0,
3,3,Male,7896,Dark,3,"Nerd Glasses,Pipe,Wild Hair",0,
4,4,Male,7332,Mid,4,"Big Shades,Earring,Goat,Wild Hair",0,


In [26]:
tipos = {'Alien':1,'Ape':2,'Zombie':3,'Male':4,'Female':5}

In [27]:
df2['type_id']= df2['types'].map(tipos)

In [28]:
piel = {'Alien':1,'Ape':2,'Zombie':3,'Albino':4,'Dark':5,'Light':6,'Mid':7}

In [29]:
df2['skin_id']= df2['skin'].map(piel)

In [30]:
df2.head()

,id,types,rarity,skin,total_traits,traits,total_hidden_traits,hidden_traits,type_id,skin_id
0,0,Female,2023,Mid,3,"Blonde Bob,Earring,Green Eye Shadow",1,Earring,5,7
1,1,Male,4352,Dark,2,"Mohawk,Smile",0,,4,5
2,2,Female,8090,Light,1,Wild Hair,0,,5,6
3,3,Male,7896,Dark,3,"Nerd Glasses,Pipe,Wild Hair",0,,4,5
4,4,Male,7332,Mid,4,"Big Shades,Earring,Goat,Wild Hair",0,,4,7


In [31]:
df2.tail()

,id,types,rarity,skin,total_traits,traits,total_hidden_traits,hidden_traits,type_id,skin_id
2553,9995,Female,2466,Albino,2,"Purple Eye Shadow,Straight Hair Dark",0,,5,4
2554,9996,Male,4323,Light,4,"Cigarette,Crazy Hair,Earring,Smile",0,,4,6
2555,9997,Zombie,498,Zombie,2,"Cap Forward,Front Beard",0,,3,3
2556,9998,Female,1271,Mid,3,"Black Lipstick,Clown Eyes Green,Wild White Hair",0,,5,7
2557,9999,Female,9233,Dark,2,"Mohawk,Nerd Glasses",0,,5,5


In [35]:
df2.set_index('id',inplace=True)

In [36]:
df2

,types,rarity,skin,total_traits,traits,total_hidden_traits,hidden_traits,type_id,skin_id
id,,,,,,,,,
0,Female,2023,Mid,3,"Blonde Bob,Earring,Green Eye Shadow",1,Earring,5,7
1,Male,4352,Dark,2,"Mohawk,Smile",0,,4,5
2,Female,8090,Light,1,Wild Hair,0,,5,6
3,Male,7896,Dark,3,"Nerd Glasses,Pipe,Wild Hair",0,,4,5
4,Male,7332,Mid,4,"Big Shades,Earring,Goat,Wild Hair",0,,4,7
...,...,...,...,...,...,...,...,...,...
9995,Female,2466,Albino,2,"Purple Eye Shadow,Straight Hair Dark",0,,5,4
9996,Male,4323,Light,4,"Cigarette,Crazy Hair,Earring,Smile",0,,4,6
9997,Zombie,498,Zombie,2,"Cap Forward,Front Beard",0,,3,3


In [38]:
df2[['type_id','rarity','skin_id','total_traits','total_hidden_traits']].to_csv(r'./to_database/punk.csv',index=True)

In [39]:
mydb = mysql.connector.connect(
    host = 'localhost',    
    user = 'manel',
    password = 'itacademy',
    database="mydb")

mycursor = mydb.cursor()


In [40]:
sql_query = 'SHOW TABLES'
mycursor.execute(sql_query)

for x in mycursor:
  print(x) 

('punk',)
('punk_skin',)
('punk_type',)
('shops',)
('traits',)
('transactions',)


In [34]:
# executo una comanda de SQL language
sql_query = 'SELECT * FROM punk_skin;'
mycursor.execute(sql_query)
myresult = mycursor.fetchmany()
for x in myresult:
  print(x)

In [35]:
# executo una comanda de SQL language
sql_query = 'SELECT * FROM punk_skin'
mycursor.execute(sql_query)
myresult = mycursor.fetchall()
print(myresult)

[]
